In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
from os import listdir
from os.path import join
import matplotlib.pyplot as plt

In [ ]:
index_dir={'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'+':10,'-':11,'times':12, '!':13, '(':14, ')':15}, '+': 16

In [ ]:
def get_index(directory):
    try:
        return index_dir[directory]
    except KeyError:
        print(f"Directory '{directory}' not found in index_dir.")
        return None

def load_images(folder):
    train_data = []

    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE) 
        #img = ~img
        if img is not None:
            img = ~img
            _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
            ctrs, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0]) 
            m = 0
            for c in cnt:
                x, y, w, h = cv2.boundingRect(c)
                m = max(w*h, m)
                if m == w*h:
                    x_max,y_max,w_max,h_max=x,y,w,h
            im_crop = img[y_max:y_max+h_max+10, x_max:x_max+w_max+10] 
            im_resize = cv2.resize(im_crop, (28, 28)) 
            im_resize = np.reshape(im_resize, (784, 1)) 
            train_data.append(im_resize)
    return train_data

In [ ]:
!unzip /content/drive/MyDrive/extracted_images.zip

In [ ]:
dataset_dir = '/content/extracted_images'
directory_list = listdir(dataset_dir)

first = True
data = []
print('Importing...')
for directory in directory_list:
    full_path = os.path.join(dataset_dir, directory)
    if os.path.isdir(full_path):
        print(directory)
        if first:
            first = False
            data = load_images(full_path)
            for i in range(len(data)):
                data[i] = np.append(data[i], [str(get_index(directory))])
            continue

        auxiliary_data = load_images(full_path)
        for i in range(len(auxiliary_data)):
            auxiliary_data[i] = np.append(auxiliary_data[i], [str(get_index(directory))])
        data = np.concatenate((data, auxiliary_data))

df = pd.DataFrame(data, index=None)
df.to_csv('./sample_data/train_data.csv', index=False)

In [ ]:
from tensorflow.keras.models import model_from_json
print('Loading Model...')
model_json = open('model/model.json', 'r')
loaded_model_json = model_json.read()
model_json.close()
model = model_from_json(loaded_model_json)

print('Loading weights...')
model.load_weights("model/model_weights.h5")

In [ ]:
labels = [ '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','+','-','x']

In [ ]:
from PIL import Image,ImageDraw,ImageGrab

In [ ]:
def activate_event(event):
        global lasx,lasy
        lasx,lasy=event.x,event.y   

In [ ]:
def draw_smth(event):
    global lasx,lasy
    cv.create_line((lasx,lasy,event.x,event.y),fill='black',width=4)
    #cv.draw.line([x1, y1, x2, y2], fill="black", width=7)
    lasx,lasy=event.x,event.y

In [ ]:
def save():
    filename="canvas.jpg"
    widget=cv
    x=root.winfo_rootx()+widget.winfo_x()+50
    y=root.winfo_rooty()+widget.winfo_y()+50
    x1=x+widget.winfo_width()
    y1=y+widget.winfo_height()
    
    ImageGrab.grab().crop((x,y,x1,y1)).save(filename)

In [ ]:
from sympy import *

class Solver:

    def __init__(self, equation):
        self.equation = str(equation)
        self.leftEqu = []

    def convertEquationIntoGeneralForm(self):

        leftSide, rightSide = '', ''
        equalIndx = self.equation.index('=')
        leftSide = self.equation[0:equalIndx]
        rightSide = self.equation[equalIndx+1:len(self.equation)]

        if rightSide[0].isalpha() or rightSide[0].isdigit():
            rightSide = '+' + rightSide

        for i in range(0, len(rightSide)):
            if rightSide[i] == '+':
                rightSide = rightSide[0:i] + '-' + rightSide[i+1:len(rightSide)]
            elif rightSide[i] == '-':
                rightSide = rightSide[0:i] + '+' + rightSide[i+1:len(rightSide)]
            leftSide += rightSide[i]

        self.equation = leftSide + '=' + '0'
        self.leftEqu = leftSide

    def solveEquation(self):

        self.convertEquationIntoGeneralForm()
        sympy_eq = sympify("Eq(" + self.equation.replace("=", ",") + ")")
        roots = solve(sympy_eq)
        
        return roots  

In [ ]:
def predictFromArray(arr):
    result = model.predict_classes(arr)
    return result    

In [ ]:
def solution():
    img = cv2.imread('canvas.jpg',cv2.IMREAD_GRAYSCALE)
    img = ~img
    ret,thresh = cv2.threshold(img,127, 255,cv2.THRESH_BINARY)
    ctrs,_ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    img_data = []
    rects = []
    for c in cnt :
        x, y, w, h = cv2.boundingRect(c)
        rect = [x, y, w, h]
        rects.append(rect)
    final_rect = [i for i in rects]
    for r in final_rect:
        x,y,w,h = r[0],r[1],r[2],r[3]
        img = thresh[y:y+h+10, x:x+w+10]
        img = cv2.resize(img, (28, 28)) 
        img = np.reshape(img, (1, 28, 28)) 
        img_data.append(img)
        
    mainEquation=[]
    operation = ''
    for i in range(len(img_data)):
                img_data[i] = np.array(img_data[i])
                img_data[i] = img_data[i].reshape(-1, 28, 28, 1)
                result=predictFromArray(img_data[i])
                i=result[0]
                mainEquation.append(labels[i])
        
    StringEquation=""
    for i in range(len(mainEquation)):
            a=mainEquation[i]
            if(a.isdigit()==False and a.isalpha()==False and i<len(mainEquation)-1):
                if(a==mainEquation[i+1]=='-'):
                    StringEquation+='='
                else:
                    StringEquation+=a
            if(a.isalpha()==True):
                if(i>0):
                    if(mainEquation[i-1].isdigit()):
                        StringEquation+="*"+a
                    else:
                        StringEquation+=a
                else:
                    StringEquation+=a
            if(a.isdigit()==True):
                if(i>0):
                    if(mainEquation[i-1].isdigit()):
                        StringEquation+=a
                    elif(mainEquation[i-1].isalpha()):
                         StringEquation+="^"+a
                    else:
                        StringEquation+=a
                else:
                    StringEquation+=a
            
    newStr=""
    l=list(StringEquation)   
    for i in range(len(l)):
            if(l[i]=="="):
                newStr=l[:i+1]+l[i+2:]
    print(newStr)
    equ=""
    for i in newStr:
            equ+=i
    solution=Solver(equ)
            
    str1=''        
    roots=solution.solveEquation()
    st=[]
    for i in roots:
        i=str(i)
        st.append(i)
       
    str1=', '.join(st)
 
    solving(equ,str1)      

In [ ]:
from tkinter import*

root=Tk()
root.resizable(0,0)
root.title('Equation Solver')

lasx,lasy=None,None

cv=Canvas(root,width=1200,height=500,bg='white')
#cv=Canvas(root,width=1600,height=500,bg='white')
cv.grid(row=0,column=0,pady=2,sticky=W,columnspan=2)
cve2=Label(root)
cve=Label(root,font=("Helvetica",16))
cve.grid(row=0, column=1,pady=1, padx=1)
cve2.grid(row=1, column=1,pady=1, padx=1)

cv.bind('<Button-1>',activate_event)
cv.bind('<B1-Motion>',draw_smth)
btn_save=Button(text="Save",command=save,bg='#6495ED',fg='White')
btn_save.grid(row=2,column=0,pady=1,padx=1)

btn_predict=Button(text="Predict",command=solution,bg='#6495ED',fg='White')
btn_predict.grid(row=2,column=1,pady=1,padx=1)


def solving(equ,roots):
    
    cve2.configure(text='Your Equation is : '+equ)
    cve.configure(text='Result : '+(roots)+'\n')    
root.mainloop()